In [ ]:
from beanie import init_beanie
from motor.motor_asyncio import AsyncIOMotorClient
from tropic.api.documents import PolymerisationDocument, MonomerDocument
from pydantic import Field

from beanie import View


class MonomerSummary(View):
    monomer_id: str = Field(alias="_id")
    has_exp: bool
    has_comp: bool

    class Settings:
        source = PolymerisationDocument
        pipeline = [
            {
                "$group": {
                    "_id": "$monomer.smiles",
                    "has_exp": {"$max": "$parameters.is_experimental"},
                    "has_comp": {"$max": {"$not": "$parameters.is_experimental"}},
                    "monomer": {"$first": "$monomer"},
                }
            },
        ]


await init_beanie(
    database=AsyncIOMotorClient("mongodb://localhost:27017")["tropic"],
    document_models=[PolymerisationDocument, MonomerSummary, MonomerDocument],
    recreate_views=True,
)
# summaries = await MonomerSummary.find(fetch_links=True).to_list()
# summaries
summaries = PolymerisationDocument.find(fetch_links=True).project(MonomerSummary)
summaries

TypeError: object FindMany can't be used in 'await' expression

In [32]:
await PolymerisationDocument.find(fetch_links=True).to_list()

[17:33:57] Initializing MetalDisconnector
[17:33:57] Running MetalDisconnector
[17:33:57] Initializing Normalizer
[17:33:57] Running Normalizer
[17:33:57] Initializing MetalDisconnector
[17:33:57] Running MetalDisconnector
[17:33:57] Initializing Normalizer
[17:33:57] Running Normalizer
[17:33:57] Initializing MetalDisconnector
[17:33:57] Running MetalDisconnector
[17:33:57] Initializing Normalizer
[17:33:57] Running Normalizer
[17:33:57] Initializing MetalDisconnector
[17:33:57] Running MetalDisconnector
[17:33:57] Initializing Normalizer
[17:33:57] Running Normalizer
[17:33:57] Initializing MetalDisconnector
[17:33:57] Running MetalDisconnector
[17:33:57] Initializing Normalizer
[17:33:57] Running Normalizer
[17:33:57] Initializing MetalDisconnector
[17:33:57] Running MetalDisconnector
[17:33:57] Initializing Normalizer
[17:33:57] Running Normalizer
[17:33:57] Initializing MetalDisconnector
[17:33:57] Running MetalDisconnector
[17:33:57] Initializing Normalizer
[17:33:57] Running Nor

[PolymerisationDocument(polymerisation_id='poly-1', type='RER', monomer=MonomerDocument(monomer_id='monomer-1', smiles='CC1OCCOC1=O', inchi='InChI=1S/C5H8O3/c1-4-5(6)8-3-2-7-4/h4H,2-3H2,1H3', molecular_weight=116.047344116, functional_groups=['[R][O][R]', 'O=[C]([R])[O][R]'], iupac_name=None, pubchem_cid=None, ring_size=6, xyz='16\n\nC      2.087875    0.209256    0.394586\nC      0.776270    0.456501   -0.298677\nO     -0.171062    1.048881    0.533741\nC     -1.422785    0.744570   -0.008567\nC     -1.762978   -0.629889    0.560572\nO     -0.783180   -1.509067   -0.014656\nC      0.240452   -0.888188   -0.717956\nO      0.783646   -1.397228   -1.738664\nH      1.993449   -0.630194    1.105212\nH      2.779138   -0.166414   -0.412230\nH      2.554818    1.103170    0.798270\nH      0.884613    1.109367   -1.171476\nH     -2.196337    1.485085    0.188336\nH     -1.353646    0.587184   -1.099788\nH     -2.779571   -0.896974    0.221963\nH     -1.630701   -0.626061    1.659334\n', id=Ob

In [ ]:
from beanie import init_beanie
from motor.motor_asyncio import AsyncIOMotorClient
from tropic.api.documents import PolymerisationDocument, MonomerDocument
from pydantic import Field, BaseModel

from beanie import View


class MonomerSummary(BaseModel):
    monomer_id: str = Field(alias="_id")
    smiles: str
    has_exp: bool
    has_comp: bool


pipeline = [
    {
        "$group": {
            "_id": "$monomer.monomer_id",
            "smiles": {"$first": "$monomer.smiles"},
            "has_exp": {"$max": "$parameters.is_experimental"},
            "has_comp": {"$max": {"$not": "$parameters.is_experimental"}},
        }
    },
    {
        "$project": {
            "_id": 0,
            "monomer_id": "$_id",
        }
    },
]

await init_beanie(
    database=AsyncIOMotorClient("mongodb://localhost:27017")["tropic"],
    document_models=[PolymerisationDocument, MonomerDocument],
    recreate_views=True,
)

summaries = (
    await PolymerisationDocument.find(fetch_links=True)
    .aggregate(
        pipeline,
        # projection_model=MonomerSummary
    )
    .to_list()
)
print(summaries)

[{'smiles': 'O=C1CCCCCCN1', 'has_exp': False, 'has_comp': True, 'monomer_id': 'monomer-12'}, {'smiles': 'S=C1CSCCS1', 'has_exp': False, 'has_comp': True, 'monomer_id': 'monomer-52'}, {'smiles': 'CC1(C)CCCC1', 'has_exp': False, 'has_comp': True, 'monomer_id': 'monomer-99'}, {'smiles': 'O=C(OCc1ccccc1)C1CC=CC1', 'has_exp': False, 'has_comp': True, 'monomer_id': 'monomer-116'}, {'smiles': 'O=C1COC(=O)CO1', 'has_exp': True, 'has_comp': False, 'monomer_id': 'monomer-154'}, {'smiles': 'O=C1OCCCCCO1', 'has_exp': False, 'has_comp': True, 'monomer_id': 'monomer-44'}, {'smiles': 'CC1(C)CC1', 'has_exp': False, 'has_comp': True, 'monomer_id': 'monomer-98'}, {'smiles': 'CC1CCCC1', 'has_exp': False, 'has_comp': True, 'monomer_id': 'monomer-96'}, {'smiles': 'O=C1SCCSC1c1ccccc1', 'has_exp': False, 'has_comp': True, 'monomer_id': 'monomer-131'}, {'smiles': 'CCCC1CCCOC1=O', 'has_exp': True, 'has_comp': False, 'monomer_id': 'monomer-167'}, {'smiles': 'CCCC1CCCC(=O)O1', 'has_exp': False, 'has_comp': True,

Depends(FilterWrapper)